# Minimal character-level Vanilla RNN model.

RNN stand for "Recurent Neural Network".  
To understand why RNN are so hot you _must_ read [this](http://karpathy.github.io/2015/05/21/rnn-effectiveness/)!  

This notebook to explain the _[Minimal character-level Vanilla RNN model](https://gist.github.com/karpathy/d4dee566867f8291f086)_ written by __Andrej Karpathy__  
This code create a RNN to generate a text, char after char, by learning char after char from a textfile.

I love this _character-level Vanilla RNN_ code because it doesn't use any library except numpy.
All the NN magic in 112 lines of code, no need to understand any dependency. Everything is there! I'll try to explain in detail every line of it.

This notebook is for real beginners who whant to understand RNN concept by reading code.  
Feedback welcome __@dh7net__
 
## Let's start!  
First let's run the whole code:

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('methamorphosis.txt', 'r').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

def lossFun(inputs, targets, hprev):
  """
  inputs,targets are both list of integers.
  hprev is Hx1 array of initial hidden state
  returns the loss, gradients on model parameters, and last hidden state
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
  # backward pass: compute gradients going backwards
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

def sample(h, seed_ix, n):
  """ 
  sample a sequence of integers from the model 
  h is memory state, seed_ix is seed letter for first time step
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  return ixes

n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0
while n<=1000: # was while True: in original code
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  if p+seq_length+1 >= len(data) or n == 0: 
    hprev = np.zeros((hidden_size,1)) # reset RNN memory
    p = 0 # go from start of data
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # sample from the model now and then
  if n % 100 == 0:
    sample_ix = sample(hprev, inputs[0], 200)
    txt = ''.join(ix_to_char[ix] for ix in sample_ix)
    print '----\n %s \n----' % (txt, )

  # forward seq_length characters through the net and fetch gradient
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001
  if n % 100 == 0: print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
  
  # perform parameter update with Adagrad
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update

  p += seq_length # move data pointer
  n += 1 # iteration counter 

data has 119163 characters, 61 unique.
----
 xrvvvQSiTpL,FvEnf,Ipcgj(EEouhUDNY AdH(t'elMgwgHye)yjOyWouEfl
uIhbf:i,vPSl(P:sw
ka,wfxaFSHDtJnj:ez;NIiv"FYu;hegJ;PSGFfehrpjt''w;Vw(LMo""DGcqcJVP"nAeMqO;dVDyEBblVM"xbOn
Lr-lmzuM
!PFlJsEOwlDqQNGrcQfw'nms 
----
iter 0, loss: 102.771834
----
 asaarus nss afdey o
a  o ostoaaen oohieel tsb es esass  r aseassua see seaofaa,ensausoaausu s o aemohshgaoou ssoefoa asdseohastate.usy   wna ieeip  oose  tooe su  oss c    darenavtsed  b soobspis wnge 
----
iter 100, loss: 102.568394
----
 .hatd neooivtw mwne g  ddsevbr ibe revnnhus.oGe si- t nrt
l vcb  dssheoo  eee nun yes uomdro aubfcywf sku wh g btsan bia akte slrkt b s  snsodobh d To yi o s o o c dod
fb ybaute bewonwsesu nilx   oi n 
----
iter 200, loss: 100.476990
----
  ve  anos g n oba  e e are ai o oet e ge   to be m fe gab e  dipot eo.e unhia
 not  d ne  a  it uag  a ghkce' ut n wert a ae t,iou aneoiw  pilhtg weu ! ant ele m eeea, t  ig  
eice teecee se teeigiven 
----
iter 300, loss: 98.186100
----

If you are not a NN expert, the code is not easy to understand.  

If you look to the results you can see that the code iterate 1000 time, calculate a __loss__ that decrease over time, and output some text each 100 itaration.
The output from the first iteration looks random.
After 1000 iterations, the NN is able to create words that have plausible size, don't use too much caps, and can create correct small words like "the", "they", "be", "to".
If you let the code learn over a nigth the NN will be able to create correct sentences. 

## Theorie
This code build a neural network that is able to predict a char from the previous one.  
It learn from a text file, so he can learn words and sentence.  
At each step it can use some results from the previous step to keep in memory what is going on.  
For instance if the previous char are "hello worl" the model can guess that the next char is "d".

This model contain parameters that are initialized randomly and the trainning phase try to find optimal values for each of them. 
During the trainning process we do a gradient descent:
* We give to the model a pair of char: the input char and the target char. The target char is the char the network should guess, it is the next char in our trainning text file.
* We calculate the probability for every possible next char according to the state of the model, using the paramters (This is the forward pass).
* We create a distance (the loss) between the previous probabilty and the target char.
* We calculate gradients for each of our parameters to see witch impact they have on the loss. (A fast way to calculate all gradients is called the backward pass).
* We update all parameters in the direction that help to minimise the loss
* We iterate until their is no more progress

# Let's dive in! 

## The code contains 4 parts
* Load the trainning data
  * and encode decode char into vectors
* Define the Network
* Define a function to create sentences from the model
* Define a loss function
  * Forward pass
  * Loss
  * Backward pass
* Train the network
  * Feed the network
  * Calculate gradiend and use Adagrad to update the model parameters
  * Output a text to see the progress of the training
 
Let's have a closer look to every line of the code.

## Load the training data

The network need a big txt file as an input.

The content of the file will be used to train the network.

For this example, I used Methamorphosis from Kafka (Public Domain).

In [2]:
"""                                                                                                                                                                                           
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)                                                                                                             
BSD License                                                                                                                                                                                   
"""
import numpy as np

# data I/O                                                                                                                                                                                    
data = open('methamorphosis.txt', 'r').read() # should be simple plain text file   

### Encode/Decode char/vector
Neural networks can only works on vectors. (a vector is an array of float)
So we need a way to encode and decode a char as a vector.

For this we count the number of unique char (*vocab_size*). It will be the size of the vector. 
The vector contain only zero exept for the position of the char wherae the value is 1.

#### First we calculate *vocab_size*:

In [3]:
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print 'data has %d characters, %d unique.' % (data_size, vocab_size)

data has 119163 characters, 61 unique.


#### Then we create 2 dictionary to encode and decode a char to an int

In [4]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }
print char_to_ix
print ix_to_char

{'\n': 0, '!': 1, ' ': 2, '"': 3, "'": 4, ')': 5, '(': 6, '-': 7, ',': 8, '.': 9, ';': 10, ':': 11, '?': 12, 'A': 13, 'C': 14, 'B': 15, 'E': 16, 'D': 17, 'G': 18, 'F': 19, 'I': 20, 'H': 21, 'J': 22, 'M': 23, 'L': 24, 'O': 25, 'N': 26, 'Q': 27, 'P': 28, 'S': 29, 'U': 30, 'T': 31, 'W': 32, 'V': 33, 'Y': 34, 'a': 35, 'c': 36, 'b': 37, 'e': 38, 'd': 39, 'g': 40, 'f': 41, 'i': 42, 'h': 43, 'k': 44, 'j': 45, 'm': 46, 'l': 47, 'o': 48, 'n': 49, 'q': 50, 'p': 51, 's': 52, 'r': 53, 'u': 54, 't': 55, 'w': 56, 'v': 57, 'y': 58, 'x': 59, 'z': 60}
{0: '\n', 1: '!', 2: ' ', 3: '"', 4: "'", 5: ')', 6: '(', 7: '-', 8: ',', 9: '.', 10: ';', 11: ':', 12: '?', 13: 'A', 14: 'C', 15: 'B', 16: 'E', 17: 'D', 18: 'G', 19: 'F', 20: 'I', 21: 'H', 22: 'J', 23: 'M', 24: 'L', 25: 'O', 26: 'N', 27: 'Q', 28: 'P', 29: 'S', 30: 'U', 31: 'T', 32: 'W', 33: 'V', 34: 'Y', 35: 'a', 36: 'c', 37: 'b', 38: 'e', 39: 'd', 40: 'g', 41: 'f', 42: 'i', 43: 'h', 44: 'k', 45: 'j', 46: 'm', 47: 'l', 48: 'o', 49: 'n', 50: 'q', 51: 'p',

#### Finaly we create a vector from a char like this:
The dictionary defined above allow us to create a vector of size 61 instead of 256.  
Here and exemple for char 'a'  
The vector contains only zero, except at position char_to_ix['a'] where we put a 1.

In [5]:
vector_for_char_a = np.zeros((vocab_size, 1))
vector_for_char_a[char_to_ix['a']] = 1
#print vector_for_char_a
print vector_for_char_a.ravel()

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.]


## Definition of the network

The neural network is made of 3 layers:
* an input layer
* an hidden layer
* an output layer

All layers are fully connected.  
The hidden layer is connected to the output and to itself: the values from an iteration are used for the next one.

To centralise values that matter for the training (_hyper parameters_) we also define the _sequence lenght_ and the _learning rate_

In [6]:
# hyperparameters                                                                                                                                                                             
hidden_size = 100 # size of hidden layer of neurons                                                                                                                                           
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [27]:
# model parameters                                                                                                                                                                            
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
print 'Wxh contain', Wxh.size, 'parameters'
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
print 'Whh contain', Whh.size, 'parameters'
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output    
print 'Why contain', Why.size, 'parameters'
bh = np.zeros((hidden_size, 1)) # hidden bias
print 'bh contain', bh.size, 'parameters'
by = np.zeros((vocab_size, 1)) # output bias
print 'by contain', by.size, 'parameters'

Wxh contain 6100 parameters
Whh contain 10000 parameters
Why contain 6100 parameters
bh contain 100 parameters
by contain 61 parameters


The model parameters are adjusted during the trainning.
* _Wxh_ are parameters to connect a vector that contain one input to the hidden layer.
* _Whh_ are parameters to connect the hidden layer to itself. This is the Key of the Rnn: Recursion is done by injecting the previous values from the output of the hidden state, to itself at the next iteration.
* _Why_ are parameters to connect the hidden layer to the output
* _bh_ contains the hidden bias
* _by_ contains the output bias

You'll see in the next section how theses parameters are used to create a sentence.

## Create a sentence from the model

In [28]:
def sample(h, seed_ix, n):
  """                                                                                                                                                                                         
  sample a sequence of integers from the model                                                                                                                                                
  h is memory state, seed_ix is seed letter for first time step                                                                                                                               
  """
  x = np.zeros((vocab_size, 1))
  x[seed_ix] = 1
  ixes = []
  for t in xrange(n):
    h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
    y = np.dot(Why, h) + by
    p = np.exp(y) / np.sum(np.exp(y))
    ix = np.random.choice(range(vocab_size), p=p.ravel())
    x = np.zeros((vocab_size, 1))
    x[ix] = 1
    ixes.append(ix)
  txt = ''.join(ix_to_char[ix] for ix in ixes)
  print '----\n %s \n----' % (txt, )

hprev = np.zeros((hidden_size,1)) # reset RNN memory  
sample(hprev,char_to_ix['a'],200)

----
 (t"?rBYcLzsnzMyNAv cWbEn?a'HaGtP:dBicF
zypJWcgCLkYICqlUBgUVafimymwYH(
bwCbC"Fm,cceBWk hyr"!kQmPBum!kpxE:'LtdTPjxEjfduN-"kQuFWU.BAaD,hPADtfLx) jO!nLJl,vk(dDbx;I
(,mmBhnzQaHbMabhB:(FPv?drB-WodnMY,vMhe'w 
----


## Define the loss function
The __loss__ is a key concept.  
It is a value that describe how bag/good is our model.  
It is always positive, the closest to zero, the better is our model.  
(A good model is a model where the predicted output is close to the training output)
  
During the trainning phase we want to minimize the loss.

The loss function calculate the loss but also the gradients (see backward pass):
* It perform a forward pass to calculate the next char given a char from the trainning set.
* It calculate the loss by comparing the predicted char to the following char of the tranning set.
* It calculate the backward pass to calculate the gradients

This function take as input:
* a list of input char
* a list of target char
* and the previous hidden state

This function output:
* the loss
* the gradient for each parameters between layers
* the last hidden state

Here the code:

In [14]:
def lossFun(inputs, targets, hprev):
  """                                                                                                                                                                                         
  inputs,targets are both list of integers.                                                                                                                                                   
  hprev is Hx1 array of initial hidden state                                                                                                                                                  
  returns the loss, gradients on model parameters, and last hidden state                                                                                                                      
  """
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  loss = 0
  # forward pass                                                                                                                                                                              
  for t in xrange(len(inputs)):
    xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation                                                                                                                        
    xs[t][inputs[t]] = 1
    hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state                                                                                                            
    ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars                                                                                                           
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars                                                                                                              
    loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)                                                                                                                       
  # backward pass: compute gradients going backwards                                                                                                                                          
  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y                                                                                                                                                     
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients                                                                                                                 
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]


### Forward pass
The forward pass use the parameters of the model (Wxh, Whh, Why, bh, by) to calculate the next char given a char from the trainning set.

xs[t] is the vector that encode the char at position t
ps[t] is the probabilities for next char

```python
hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
```

or is dirty pseudo code for each char
```python
hs = input*Wxh + last_value_of_hidden_state*Whh + bh
ys = hs*Why + by
ps = normalized(ys)
```

To dive into the code, we'll work on one char only (we set t=0 ; instead of the for each).

In [9]:
# uncomment the print to get some details
xs, hs, ys, ps = {}, {}, {}, {}
hs[-1] = np.copy(hprev)
# forward pass                                                                                                                                                                              
t=0 # for t in xrange(len(inputs)):
xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
xs[t][inputs[t]] = 1 
# print xs[t]
hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state 
ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
# print ys[t]
ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars  
print ps[t].ravel()

[ 0.01651179  0.01642741  0.01629968  0.01645583  0.01644896  0.01625604
  0.01623776  0.01654136  0.01644857  0.01626252  0.01596373  0.01613876
  0.01675783  0.01614578  0.01673236  0.01665156  0.01640061  0.01628567
  0.01613562  0.01648003  0.01602888  0.01634054  0.01661922  0.01652779
  0.01631034  0.01597116  0.01649952  0.01634823  0.0165061   0.01659232
  0.01663144  0.01658593  0.01651304  0.01639281  0.01646073  0.01651658
  0.0164507   0.01638602  0.01622293  0.01659311  0.0164573   0.01637815
  0.01633373  0.01640152  0.01665601  0.01648789  0.01649384  0.01608145
  0.01625795  0.01657984  0.01641883  0.0163816   0.01622493  0.01644234
  0.01641011  0.01624322  0.01615029  0.01623067  0.01612011  0.0167561
  0.0164149 ]


You can run the previous code several time. A probability is generated during each iteration.

In [10]:
# Let's build a dict to see witch probablity is associated with witch char
probability_per_char =  { ch:ps[t].ravel()[i] for i,ch in enumerate(chars) }
# uncoment the next line to see the raw result
# print probability_per_char

# To print the probability in a way that is more easy to read.
for x in range(vocab_size):
    print 'p(' + ix_to_char[x] + ")=", "%.4f" % ps[t].ravel()[x],
    if (x%7==0):
        print ""
    else:
        print "",

p(
)= 0.0165 
p(!)= 0.0164  p( )= 0.0163  p(")= 0.0165  p(')= 0.0164  p())= 0.0163  p(()= 0.0162  p(-)= 0.0165 
p(,)= 0.0164  p(.)= 0.0163  p(;)= 0.0160  p(:)= 0.0161  p(?)= 0.0168  p(A)= 0.0161  p(C)= 0.0167 
p(B)= 0.0167  p(E)= 0.0164  p(D)= 0.0163  p(G)= 0.0161  p(F)= 0.0165  p(I)= 0.0160  p(H)= 0.0163 
p(J)= 0.0166  p(M)= 0.0165  p(L)= 0.0163  p(O)= 0.0160  p(N)= 0.0165  p(Q)= 0.0163  p(P)= 0.0165 
p(S)= 0.0166  p(U)= 0.0166  p(T)= 0.0166  p(W)= 0.0165  p(V)= 0.0164  p(Y)= 0.0165  p(a)= 0.0165 
p(c)= 0.0165  p(b)= 0.0164  p(e)= 0.0162  p(d)= 0.0166  p(g)= 0.0165  p(f)= 0.0164  p(i)= 0.0163 
p(h)= 0.0164  p(k)= 0.0167  p(j)= 0.0165  p(m)= 0.0165  p(l)= 0.0161  p(o)= 0.0163  p(n)= 0.0166 
p(q)= 0.0164  p(p)= 0.0164  p(s)= 0.0162  p(r)= 0.0164  p(u)= 0.0164  p(t)= 0.0162  p(w)= 0.0162 
p(v)= 0.0162  p(y)= 0.0161  p(x)= 0.0168  p(z)= 0.0164 


In [11]:
# We can create the next char from the above distribution
ix = np.random.choice(range(vocab_size), p=ps[t].ravel())
print "Next char is :", ix_to_char[ix]

Next char is : D


But during the trainning process, we don't want to calculate char,  
We want to calculate a loss to mesure how good/bad our model is.

### Loss
For each char in the input the forward pass calculate the probability of the next char  
The loss is the sum 
```python
loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
```

The loss is calculate using Softmax. [more info here](https://jamesmccaffrey.wordpress.com/2013/11/05/why-you-should-use-cross-entropy-error-instead-of-classification-error-or-mean-squared-error-for-neural-network-classifier-training/) and [here](https://en.wikipedia.org/wiki/Softmax_function).


In [12]:
print 'Next char from training (target) was number', targets[t], 'witch is "' + ix_to_char[targets[t]] + '"'
print 'Probability for this letter was', ps[t][targets[t],0]

loss = -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
print 'loss for this input&target pair is', loss

Next char from training (target) was number 36 witch is "c"
Probability for this letter was 0.0164507007455
loss for this input&target pair is 4.10738720417


### Backward pass

The naive way to calculate all gradients would be to recalculate a loss for small variations for each parameters.
This is possible but super time consuming.
There is a technics to calculates all the gradients for all the parameters at once, the backdrop propagation.  
Gradients are calculated in the oposite order of the forward pass, using simple technics.
[Here](http://karpathy.github.io/neuralnets/) a great source to understand this technic in detail.

#### goal is to calculate gradients for the forward formula:
hs = input*Wxh + last_value_of_hidden_state*Whh + bh  
ys = hs*Why + by

dby = 


In [13]:
# backward pass: compute gradients going backwards                                                                                                                                          
dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
dbh, dby = np.zeros_like(bh), np.zeros_like(by)
dhnext = np.zeros_like(hs[0])
t=0 #for t in reversed(xrange(len(inputs))):
dy = np.copy(ps[t])
dy[targets[t]] -= 1 # backprop into y   
print dy.ravel()
dWhy += np.dot(dy, hs[t].T)
print dWhy.ravel()
dby += dy
print dby.ravel()
dh = np.dot(Why.T, dy) + dhnext # backprop into h                                                                                                                                         
dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity                                                                                                                     
dbh += dhraw
dWxh += np.dot(dhraw, xs[t].T)
dWhh += np.dot(dhraw, hs[t-1].T)
dhnext = np.dot(Whh.T, dhraw)
for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
  np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients     

[ 0.01651179  0.01642741  0.01629968  0.01645583  0.01644896  0.01625604
  0.01623776  0.01654136  0.01644857  0.01626252  0.01596373  0.01613876
  0.01675783  0.01614578  0.01673236  0.01665156  0.01640061  0.01628567
  0.01613562  0.01648003  0.01602888  0.01634054  0.01661922  0.01652779
  0.01631034  0.01597116  0.01649952  0.01634823  0.0165061   0.01659232
  0.01663144  0.01658593  0.01651304  0.01639281  0.01646073  0.01651658
 -0.9835493   0.01638602  0.01622293  0.01659311  0.0164573   0.01637815
  0.01633373  0.01640152  0.01665601  0.01648789  0.01649384  0.01608145
  0.01625795  0.01657984  0.01641883  0.0163816   0.01622493  0.01644234
  0.01641011  0.01624322  0.01615029  0.01623067  0.01612011  0.0167561
  0.0164149 ]
[ 0.00054093 -0.00129342  0.00028735 ...,  0.00041503 -0.00423059
  0.00015588]
[ 0.01651179  0.01642741  0.01629968  0.01645583  0.01644896  0.01625604
  0.01623776  0.01654136  0.01644857  0.01626252  0.01596373  0.01613876
  0.01675783  0.01614578  0.016

## Training

In the main loop we will:
* Feed the network with portion of the file. Size of cunck is *seq_lengh*
* Use the loss function to:
  * Do forward pass to calculate all parameters for the model for a given input/output pairs
  * Do backward pass to calculate all gradiens
* Print a sentence from a random seed using the parameters of the network
* Update the model using the Adaptative Gradien technique Adagrad

### Feed the loss function with inputs and targets

We create two array of char from the data file,
the targets one is shifted compare to the inputs one.

For each char in the input array, the target array give the char that follows.

In [16]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print "inputs", inputs
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print "targets", targets

inputs [25, 49, 38, 2, 46, 48, 53, 49, 42, 49, 40, 8, 2, 56, 43, 38, 49, 2, 18, 53, 38, 40, 48, 53, 2]
targets [49, 38, 2, 46, 48, 53, 49, 42, 49, 40, 8, 2, 56, 43, 38, 49, 2, 18, 53, 38, 40, 48, 53, 2, 29]


### Adagrad to update the parameters

The easiest technics to update the parmeters of the model is this:

```python
param += dparam * step_size
```
Adagrad is a more efficient technique where the step_size are getting smaller during the training.

It use a memory variable that grow over time:
```python
mem += dparam * dparam
```
and use it to calculate the step_size:
```python
step_size = 1./np.sqrt(mem + 1e-8)
```
In short:
```python
mem += dparam * dparam
param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update 
```

### Smooth_loss

Smooth_loss doesn't play any role in the training.
It is just a low pass filtered version of the loss:
```python
smooth_loss = smooth_loss * 0.999 + loss * 0.001
```

It is a way to average the loss on over the last iterations to better track the progress


### So finally
Here the code of the main loop that does both trainning and generating text from times to times:

In [17]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad                                                                                                                
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0                                                                                                                        
while n<=1000*100:
  # prepare inputs (we're sweeping from left to right in steps seq_length long)
  # check "How to feed the loss function to see how this part works
  if p+seq_length+1 >= len(data) or n == 0:
    hprev = np.zeros((hidden_size,1)) # reset RNN memory                                                                                                                                      
    p = 0 # go from start of data                                                                                                                                                             
  inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
  targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

  # forward seq_length characters through the net and fetch gradient                                                                                                                          
  loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
  smooth_loss = smooth_loss * 0.999 + loss * 0.001

  # sample from the model now and then                                                                                                                                                        
  if n % 1000 == 0:
    print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
    sample(hprev, inputs[0], 200)

  # perform parameter update with Adagrad                                                                                                                                                     
  for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
    mem += dparam * dparam
    param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update                                                                                                                   

  p += seq_length # move data pointer                                                                                                                                                         
  n += 1 # iteration counter            

iter 0, loss: 102.771844
----
 iyJYy:
jlO.h  yANlAHYm"wmr?GbF S:VpJga,pA:mzJOEY
NhVho JWze.jEVI:?kFejuxn!"whmQ-Ys?zT,zJ"cF
Wza
MmbO,Sn
Fky(SBcIODu) ruYkEzqQdTGEVETE(x'TLIuFSdMzxPzDsgoefynCQQY?mEa)mJd,skWLgkh
vM NYg
sYut ye-PETbmaE- 
----
iter 1000, loss: 83.755345
----
 an one, wero sesu r. we 
alu nese caghe ghein be hhamltll inn ta
goarrus ane  t s fo aik. ef bhe wne navpe e gwethe nte fcp heson;  on eotk swe thh t  cog hug reh thut tho hatos  wal mheen anr  a?ny s 
----
iter 2000, loss: 68.699145
----
 
yo No holveabat ot., morhtop
thi, anreawastathiy hor the wor foghanghas
pler 
woy. oord whot thousgoily at loranghe shat rork angeveope ne pistann than toshevuod hooncasshat nathond thoAnge af tita h 
----
iter 3000, loss: 60.579479
----
 oals SOrreybit womis awewiveeont oxllnet dory ham coser nos doateyeveawaod and
oois hud. ot tallyiclisd thas
wid't af
mowiwoy somedpiswor souln,
to could fobapcere wamsow woly wiclyid wasinoven in!srt 
----
iter 4000, loss: 56.301380
----
 e fyite, 

Feedback welcome __@dh7net__!